In [128]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


def city_pop(city):
    # base url for wikipedia
    base_url = "https://en.wikipedia.org/wiki/"
    # build wikipedia link with city name for that I want to get the population details
    search = base_url + city
    # save request from website in response
    response = requests.get(search)
    # use html parser to parse requested website and save as soup
    soup = BeautifulSoup(response.content, "html.parser")
    #find all tr tags with class mergedtoprow that contain info from infobox
    infobox = soup.find_all('tr', {'class':'mergedtoprow'})
    # initialize dictionary
    new_dict = {}
    # loop through all tr items in infobox and search for the one that contains the population
    for i in infobox:
        if "Population" in i.text:
        # loop through the next siblings after population that contain the numbers
            for sibling in i.next_siblings:
                # stop the code if the class of the siblings is not mergedrow - then we reached the next category that we are not interested in
                if (sibling['class'] != ['mergedrow']):   
                    break
                # save text in th element in new_key, strip = True already removes whitespace
                new_key = sibling.select('th')[0].get_text(strip = True)
                #replace more unwanted characters
                new_key = new_key.replace('•\xa0','').replace('•','')
                # save text from td element in new_value
                new_value = sibling.select('td')[0].get_text(strip = True)
                new_value = new_value.replace('\xa0',' ')
                # save new_value with new_key in the dictionary
                new_dict[new_key] = new_value
                
    #create DataFrame from dictionary
    pop = pd.DataFrame.from_dict(new_dict, orient='index')                      
    pop = pop.reset_index()
    pop.columns = ['Description', 'Population']        
    return(pop)
            

In [132]:
city_pop('Hamburg')

,Description,Population
0,City,"1,845,229"
1,Density,"2,400/km2(6,300/sq mi)"
2,Urban,"2,484,800[1]"
3,Metro,"5,107,429"
